# POETRY GENERATION USING LSTM NETWORKS (CHARACTER-LEVEL MODEL)

###POETRY GENERATION IS A TEXT GENERATION PROBLEM THAT DEALS WITH SEQUENTIAL DATA.
###Sequential data can be defined as folows:



###Any data which has two events occurring in a particular time frame and the occurrence of event A before event B is an entirely different scenario as compared to the occurrence of event A after event B.

###In traditional ML problems, the order of occurrence of data points is not important. However in sequence prediction problems such as character-by-character text generation, our model has to output the correct character every time otherwise whole sequences of characters preceding it may become meaningless.


###This is what makes text/poetry generators tricky and to solve this problem we will use LSTM (Long Short Term Memory) networks which are a special kind of RNN's. LSTM's are well-suited to the task of capturing and remembering long-term dependencies. They are excellent at capturing context in sequential data and remembering it for long periods of time.

###In this notebook, we will train an LSTM using Keras on a collection of sonnets by Robert Frost (the famous American poet) which includes works like:
###*The Road Not Taken*
###*Mending Wall*
###*Nothing Gold Can Stay* 
###among others.
![alt text](https://revbrocoach.files.wordpress.com/2014/03/two-roads-robert-frost.jpg)

In [0]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/
%cd cs231n/
%cd poetry_generator

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive
/content/drive/My Drive/cs231n
/content/drive/My Drive/cs231n/poetry_generator


##IMPORTING LIBRARIES

In [0]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import RNN
from keras.utils import np_utils

Using TensorFlow backend.


##LOADING DATA

In [0]:
text = (open("robert_frost.txt").read())
text = text.lower()


##MAPPING CHARACTERS TO NUMBERS

In [0]:
# since machines understand numbers better than words or characters, here we will map unique character to an arbitrary unique number to speed up training
characters = sorted(list(set(text)))

n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

## PREPROCESSING OUR DATA

In [0]:

X = [] # train array
Y = [] # target array
length = len(text) # total number of characters in text
seq_length = 100 # length of sequence of characters to consider before outputting next char in sequence

# creating sequences of 100 chars each stored in sequence variable
# label stores the next/correct prediction of each sequence or the 101st value in every sequence 
for i in range(0, length-seq_length, 1):
    sequence = text[i:i + seq_length]
    label =text[i + seq_length]
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])

In [0]:
# LSTM requires training data to be of shape (number_of_sequences, length_of_sequence, number_of_features) hence reshape X
X_modified = np.reshape(X, (len(X), seq_length, 1))
# scale X for faster training
X_modified = X_modified / float(len(characters))
# one-hot encode Y to prevent any logical issues arising from mapping characters to numbers
Y_modified = np_utils.to_categorical(Y)

##DEFINING LSTM MODEL

In [0]:
# Network has 3 layers of 700 neurons each with Dropout set to 20%
model = Sequential()
model.add(LSTM(700, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(700, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(700))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

## TRAIN MODEL

In [0]:
# training code -- use pre-trained weights instead as this may take upwards of 12 hours on a Tesla K80 equivalent GPU when epoch = 100
#model.fit(X_modified, Y_modified, epochs=100, batch_size=100)
#model.save_weights('frostweights.h5')

##LOAD PRE-TRAINED WEIGHTS

In [0]:
model.load_weights('weights.h5')


##GENERATE TEXT

In [0]:

# select random row from X which is sequence of 100 characters
# predict next 100 characters following this row
string_mapped = X[99]
full_string = [n_to_char[value] for value in string_mapped]
# generating total 2000 characters
for i in range(2000):
    #reshape input to previous shape and rescale to original scale
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(characters))

    # select character (number) with max probability
    pred_index = np.argmax(model.predict(x, verbose=0))
    # convert integers back to characters
    seq = [n_to_char[value] for value in string_mapped]
    
    # remove first character from 100 character string array and append newly predicted character
    full_string.append(n_to_char[pred_index])
    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]

## PRINT GENERATED RHYMES (2000 CHARACTERS)

In [0]:
#combining text
txt=""
for char in full_string:
  txt = txt+char
  
#print generated text
print(txt)


g i stood
and looked down one as far as i could
to where it bent in the undergrowth; 

then took the one less traveles had seen he chen't recognize,
but welcomed for the place. 
this he sat listening to till she tound of feet
when far away an interrupted cry
came over houses from another street,

but not to call me back or say good-bye on meet ald stam fected to with sawdust
and swollen tight and buried under snow.
the nean see how the house as far as i could
the woods and frozen lake
the dourap and hels of cole
that soulde
to each other 
till well toward nownt have up hope
of getting home again because
he couldn't climb that slippery slope
(two miles it was) to his abode.

sometimes as an authority
on motor-cars, i'm asked if i
should say our stock was petered out of here
to say that for eestruction ice
is also great
and would suffice.

nt mother woold ho befn the stairs, the swo stiel of them all aglitter,
and birds that joined in the excited fun
by doubling and redoubling song and t

###AS WE CAN SEE, THE ALGORITHM DID START GRASPING THE STRUCTURE OF ROBERT FROST'S POETRY AND MANAGED TO PRODUCE A SOMEWHAT DECENT/COHERENT COLECTION OF RHYMES. 
###THE REAL PROBLEM WAS THAT WE WERE GPU LIMITED TO ONLY 30 EPOCH'S OVER THE TRAINING SET. THIS IS WHY WE SEE SEVERAL SPELLING MISTAKES IN THE GENERATED TEXT. THE MODEL WAS TRAINED ON A TESLA K80 GPU INSIDE GOOGLE COLAB AND I DID NOT HAVE ENOUGH TIME TO TRAIN IT FOR THE FULL 100 EPOCH'S OVER THE TRAINING DATA.
###A MORE POWERFUL GPU WITH MORE TRAINING HOURS WILL RESULT IN VERY ACCURATE, BEAUTIFUL RHYMES.